In [15]:
from http.client import responses

import vertexai

from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool

from IPython.display import Markdown

REGION = "asia-south1"
vertexai.init(location=REGION)

EMBEDDING_MODEL = "publishers/google/models/text-embedding-005"
embedding_model_config = rag.EmbeddingModelConfig(publisher_model=EMBEDDING_MODEL)

#1.Create a RAG corpus that uses an Embedding model
#The corpus will use the embedding model to encode documents and queries into vector representations that capture the semantic meaning of the documents.
# This is what will allow the system to perform efficient similarity searches
# by comparing embeddings, ensuring that the most relevant
# pieces of information from the corpus are retrieved.

rag_corpus = rag.create_corpus(
    display_name="RAG Corpus",
    embedding_model_config=embedding_model_config,
)
#rag.list_corpora()
#print(rag_corpus)

#2.Upload file to the RAG corpus
rag_file = rag.upload_file(
    corpus_name=rag_corpus.name,
    path="../data/cobbler.txt",
    display_name="cobbler.txt",
    description="Ethan, the cobbler",
)

print(rag_file)

#3.Perform direct retrieval to check RAG engine. This response isn't from Gemini model
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,
        )
    ],
    text="Who was Ethan",
    similarity_top_k=10,
    vector_distance_threshold=0.5,
)

#4.Integrate RAG retrieval tool that generates a response from the corpus
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_corpora=[rag_corpus.name],
            similarity_top_k=10,
            vector_distance_threshold=0.5,
        ),
    )
)

#5.Integrate retrieval tool with Gemini to create a RAG-Gemini model
rag_gemini_model = GenerativeModel(
    "gemini-2.5-flash",
    tools=[rag_retrieval_tool]
)

response = rag_gemini_model.generate_content("Who was Ethan?")

display(Markdown(response.text))

RagFile(name='projects/1035640742842/locations/asia-south1/ragCorpora/4172585054758764544/ragFiles/5621115715408636961', display_name='cobbler.txt', description='Ethan, the cobbler')


Ethan was a meticulous and humble cobbler who lived in a small, bustling town. He had a modest workshop and was known for repairing shoes with care and sincerity. He once received a magical golden thread that replenished his supplies as long as he continued his craft with honest effort.